In [32]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime
from tqdm import tqdm

In [33]:
json_file = open("API_key.json")
API_KEY = json.load(json_file)["API_key"]
json_file.close()

watcher = LolWatcher(API_KEY)

### Seconds to hours, minutes and seconds

In [34]:
def sec_to_hms(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
      
    return "%02d minutes %02d seconds" % (minutes, seconds)

### Get puuid of a user

In [35]:
def get_puuid(summoner_name, region='euw1'):
    puuid = watcher.summoner.by_name(region, summoner_name)["puuid"]
    return puuid

### Get item name from ID

In [36]:
def get_item_name(item_id):
    item_version = watcher.data_dragon.versions_for_region('euw1')['n']['item']
    item_name = watcher.data_dragon.items(item_version)['data'][str(item_id)]['name']
    return item_name

### Add data for a specific champion from the history of a user

In [40]:
def add_data(Summoner_name,target_champ,num_matches=50,region='euw1',REGION_full='europe'):

    # Summoner_name : 
    # num_matches :
    # locales : Europe = 'euw1','europe'
    #           Korea = 'kr','asia'
    
    player_puuid = get_puuid(Summoner_name,region)
    my_matches = watcher.match.matchlist_by_puuid(REGION_full,player_puuid,count=num_matches)

    start_time = time.time()
    date = datetime.today().strftime('%d/%m/%Y')

    row = []
    champ_list = []
    item_list = []

    df = pd.read_csv('Lol_data.csv')

    print("If the loading stops, it means that the API has reached the maximum number of requests.\n"+
          "You have to wait 2 minute before the loading resumes.\n"+
        "\nAdding "+target_champ+" data of the player "+Summoner_name+" :\n")

    for L in tqdm(range(num_matches)):

        for i in range(10): 
            try:
                test_match = watcher.match.by_id(REGION_full, my_matches[L])
            except:
                pass
            champion = test_match['info']['participants'][i]['championName']
            champ_list = np.append(champ_list,champion) # get the list of champions in the game
    
        patch = test_match['info']['gameVersion'].split(".")
        patch_str = patch[0]+'.'+patch[1]
    
        if target_champ in champ_list: # check if target champion is in the game
            target_champ_idx = np.where(champ_list == target_champ)[0][0]

            if target_champ_idx < 5: # get only the enemies 
                enemy_list = champ_list[5:10]
            else:
                enemy_list = champ_list[0:5]

            row = np.insert(enemy_list,0,target_champ)

            for k in range(6): # get the items of the target champion
                    try:
                        item_n_str = 'item'+str(k)
                        item = get_item_name(test_match['info']['participants'][target_champ_idx][item_n_str]) 
                        row = np.append(row,item) 

                    except Exception as error:
                        row = np.append(row,'no item')

            row = np.append(row,[player_puuid,my_matches[L],date,patch_str])

        if len(row):
            df_len = len(df)
            df.loc[df_len] = row

        champ_list = []
        row = []

    df.to_csv('Lol_data.csv',index=False) # adding data to csv, encoding cp1252 for specific characters

    end_time = time.time()-start_time
    print("\nData added in "+sec_to_hms(end_time)+'\n')

In [38]:
summoners_history_euw = ['Nortu','Gudule','Akn Hanima','TokÌta Ohma','Bechamelle','MENTALLY 1MMUNE','LFT BRK TR','Ping An Xi Lè',
             'EG PEIWAN Sao','HMPeiWan Yorkie','BOI NEXT DOOR','I am N1Ty','amongus potion12','no i do not care',
             'Zzzzzbr','Doope','sokrfd','Flynn 7']

summoners_history_kr = ['아 가 리','tenten12qq','이걸 세트가','ren sheng ru xi','칸야블','첼까지닉변안함','전승트','하이눈 마이','카시스 페델리안',
             '세트말고할거없음','예전에 내가 아냐','평화로운 친구','kamuE','옐로우핀','세트의 꿈','겜좀집중해서하자','남장구',
             '문신 덩크충','JYPBoss','Lestiny']

summoners = ['dopa farter','HMPeiWan NanBao','StDaimoniac','Chen Noll','TwTv NMB Jordan']
target_champ = 'Sett'
num_matches = 50

for names in summoners:
    add_data(names,target_champ,num_matches,'euw1','europe')

If the loading stops, it means that the API has reached the maximum number of requests.
You have to wait 2 minute before the loading resumes.

Adding Sett data of the player dopa farter :



  2%|█▋                                                                                 | 1/50 [00:01<01:26,  1.77s/it]

'0'


 42%|██████████████████████████████████▍                                               | 21/50 [04:02<11:51, 24.52s/it]

'0'


...

'0'


 82%|████████████████████████████████████████████████████████████████▊              | 41/50 [08:02<03:42, 24.77s/it]

'0'


100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [10:01<00:00, 12.03s/it]


Data added in 10 minutes 01 seconds
